<a href="https://colab.research.google.com/github/SSenitha/CCS3052_Advance_DSA/blob/Aashi/BellmaFord_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Dataset

In [34]:
# Load data from the repository
!wget https://raw.githubusercontent.com/SSenitha/CCS3052_Advance_DSA/refs/heads/main/Cities_of_SriLanka.csv

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [35]:
#Import necessary libraries

import math, time, heapq
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import stats
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

from collections import defaultdict
from sklearn.neighbors import BallTree

In [36]:
# Read the dataset and output the total count
#df = pd.read_csv('/content/Cities_of_SriLanka.csv')

#----------------For VS Code----------------
df = pd.read_csv('./Cities_of_SriLanka.csv')
#-------------------------------------------
print(f"Total rows count: {df['city id'].count()}")
df.rename(columns={"city id": "city_id"}, inplace=True)
df.head()

Total rows count: 2155


,city_id,district_id,name_en,name_si,name_ta,sub_name_en,sub_name_si,sub_name_ta,postcode,latitude,longitude
0,1,1,Akkaraipattu,අක්කරපත්තුව,அக்கரைப்பற்று,NULL,NULL,NULL,32400.0,7.218428,81.854116
1,2,1,Ambagahawatta,අඹගහවත්ත,அம்பகஹவத்த,NULL,NULL,NULL,90326.0,7.301756,81.674729
2,3,1,Ampara,අම්පාර,அம்பாறை,NULL,NULL,NULL,32000.0,7.301756,81.674729
3,4,1,Bakmitiyawa,බක්මිටියාව,பக்மிடியாவ,NULL,NULL,NULL,32024.0,7.029632,81.680205
4,5,1,Deegawapiya,දීඝවාපිය,தீகவாபி,NULL,NULL,NULL,32006.0,7.301756,81.674729


In [37]:
#Remove the Cities with little to no geographical difference

df = df.drop_duplicates(subset=['latitude', 'longitude'], keep='first').reset_index(drop=True)
print(f"Total rows count after dropping duplicates: {df['city_id'].count()}")

Total rows count after dropping duplicates: 1919


In [38]:
#Dropping the columns that won't be used in the making of graph/Matrix
df = df.drop(columns=["district_id","name_si","name_ta","sub_name_en","sub_name_si","sub_name_ta","postcode"])

In [39]:
def resetIndex():
  df.drop(columns=["city_id"], inplace=True)
  df.insert(0, "city_id", df.index)
  return df

In [40]:
resetIndex()
df.head()

,city_id,name_en,latitude,longitude
0,0,Akkaraipattu,7.218428,81.854116
1,1,Ambagahawatta,7.301756,81.674729
2,2,Bakmitiyawa,7.029632,81.680205
3,3,Digamadulla Weeragoda,7.390125,81.696588
4,4,Dorakumbura,7.358870,81.301428


#Adjacency/Sparse Matrix Approach

In [41]:
import math

def dist(lat1, lon1, lat2, lon2):
    #Haversine formula for spherical distances
    R = 6371.0088

    phi1 = math.radians(lat1); phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1); dlambda = math.radians(lon2 - lon1)

    a = math.sin(dphi/2.0)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda/2.0)**2
    a = min(1.0, max(0.0, a))
    return 2 * R * math.asin(math.sqrt(a))


'''
def dist(lat1, lon1, lat2, lon2):
    R = 6371   # Radius of Earth

    # Convert degrees into radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    #Calculate the linear distance between two points
    distance = dlat ** 2 + dlon ** 2
    distance = math.sqrt(distance)

    # Calculate spherical distance
    a = math.asin(distance/(2*R))
    c = R * 2 * math.asin(math.sqrt(a))

    return c
    '''


'\ndef dist(lat1, lon1, lat2, lon2):\n    R = 6371   # Radius of Earth\n\n    # Convert degrees into radians\n    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])\n\n    dlat = lat2 - lat1\n    dlon = lon2 - lon1\n\n    #Calculate the linear distance between two points\n    distance = dlat ** 2 + dlon ** 2\n    distance = math.sqrt(distance)\n\n    # Calculate spherical distance\n    a = math.asin(distance/(2*R))\n    c = R * 2 * math.asin(math.sqrt(a))\n\n    return c\n    '

In [42]:
from scipy.sparse import csr_matrix

num_cities = len(df)
k = 6

# Fit the model on the geographical coordinates
X = df[['latitude', 'longitude']]
nn = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(X)

# Find the 5 nearest neighbors
distances, indices = nn.kneighbors(X)

rows, cols, data = [], [], []
seen_edges = set()

for i in range(num_cities):
    for j in range(1, k):
        n_index = indices[i, j]

        #check if updated before
        edge = frozenset((i, n_index))
        if edge in seen_edges:
            continue
        seen_edges.add(edge)

        #If not calc distance
        n_distance = distances[i, j]

        #Add data in both ways to make it undirected
        rows.append(i); cols.append(n_index); data.append(n_distance)
        rows.append(n_index); cols.append(i); data.append(n_distance)

sparse_matrix = csr_matrix((data, (rows, cols)), shape=(num_cities, num_cities))

print("Shape of the sparse adjacency matrix:", sparse_matrix.shape)

Shape of the sparse adjacency matrix: (1919, 1919)


In [43]:
print("\nFirst 10 rows and their non-zero entries (neighbors and distances):")

for i in range(min(10, num_cities)):
    row_data = sparse_matrix.getrow(i)
    print(f"Row {i}: {list(zip(row_data.indices, row_data.data))}")


First 10 rows and their non-zero entries (neighbors and distances):
Row 0: [(np.int32(19), np.float64(0.10280801203432574)), (np.int32(20), np.float64(0.07630243464241533)), (np.int32(32), np.float64(0.09008373774760883)), (np.int32(33), np.float64(0.10312854912632105)), (np.int32(237), np.float64(0.025839241107277235)), (np.int32(256), np.float64(0.019672846708092478))]
Row 1: [(np.int32(3), np.float64(0.0910322710384622)), (np.int32(15), np.float64(0.09149139670707118)), (np.int32(24), np.float64(0.07909985753767391)), (np.int32(34), np.float64(0.07288142004886898)), (np.int32(857), np.float64(0.07483869999999992))]
Row 2: [(np.int32(6), np.float64(0.10433288545808693)), (np.int32(13), np.float64(0.18960060676363566)), (np.int32(14), np.float64(0.16315676726547387)), (np.int32(35), np.float64(0.14410087188900797)), (np.int32(736), np.float64(0.15465547323990886)), (np.int32(1287), np.float64(0.14722019989906115))]
Row 3: [(np.int32(1), np.float64(0.0910322710384622)), (np.int32(5), 

Bellman - Ford Implementation

In [44]:
# Make sure required libs are imported
import math
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# ---------- Haversine (km) ----------
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0088  # Earth's radius in km
    phi1 = math.radians(lat1); phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1); dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2.0)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda/2.0)**2
    a = min(1.0, max(0.0, a))  # safety
    return 2 * R * math.asin(math.sqrt(a))

# ---------- Build k-NN graph with Haversine distances ----------
def build_knn_graph_km(df, k=6, algorithm='ball_tree'):
    X = df[['latitude', 'longitude']].values
    num_nodes = len(df)
    nn = NearestNeighbors(n_neighbors=k, algorithm=algorithm).fit(X)
    _, indices = nn.kneighbors(X)

    rows, cols, data = [], [], []
    seen_edges = set()

    for i in range(num_nodes):
        for j in range(1, k):
            n_index = int(indices[i, j])
            edge = frozenset((i, n_index))
            if edge in seen_edges:
                continue
            seen_edges.add(edge)

            lat1, lon1 = float(df.iloc[i]['latitude']), float(df.iloc[i]['longitude'])
            lat2, lon2 = float(df.iloc[n_index]['latitude']), float(df.iloc[n_index]['longitude'])
            d_km = haversine_km(lat1, lon1, lat2, lon2)

            rows.append(i); cols.append(n_index); data.append(d_km)
            rows.append(n_index); cols.append(i); data.append(d_km)

    return csr_matrix((data, (rows, cols)), shape=(num_nodes, num_nodes)), num_nodes

# ---------- Bellman-Ford ----------
def build_edge_list_from_sparse(sparse_mat):
    coo = sparse_mat.tocoo()
    return [(int(u), int(v), float(w)) for u, v, w in zip(coo.row, coo.col, coo.data)]

def bellman_ford(num_nodes, edges, start):
    INF = float('inf')
    dist = [INF] * num_nodes
    pred = [None] * num_nodes
    dist[start] = 0.0

    for _ in range(num_nodes - 1):
        updated = False
        for u, v, w in edges:
            if dist[u] != INF and dist[u] + w < dist[v]:
                dist[v] = dist[u] + w
                pred[v] = u
                updated = True
        if not updated:
            break

    for u, v, w in edges:
        if dist[u] != INF and dist[u] + w < dist[v]:
            raise RuntimeError("Negative-weight cycle detected")

    return dist, pred

def reconstruct_path(pred, start, target):
    path = []
    cur = target
    while cur is not None:
        path.append(cur)
        if cur == start:
            break
        cur = pred[cur]
    path.reverse()
    return path if path[0] == start else []

# ---------- Shortest path wrapper ----------
def shortest_path_between(df, sparse_matrix_km, start_name, target_name):
    start_matches = df.index[df['name_en'] == start_name].tolist()
    target_matches = df.index[df['name_en'] == target_name].tolist()
    if not start_matches or not target_matches:
        raise ValueError("One or both city names not found")

    start, target = start_matches[0], target_matches[0]
    edges = build_edge_list_from_sparse(sparse_matrix_km)
    dist, pred = bellman_ford(len(df), edges, start)

    if dist[target] == float('inf'):
        print("No path found.")
        return

    path_idx = reconstruct_path(pred, start, target)
    path_names = [df.loc[i, 'name_en'] for i in path_idx]

    print(f"\nShortest path from {start_name} to {target_name}:")
    for i, city in enumerate(path_names, 1):
        print(f"  {i}. {city}")
    print(f"Total distance: {dist[target]:.3f} km")

# ---------- MAIN PROGRAM ----------
# Build the graph
sparse_matrix_km, num_nodes = build_knn_graph_km(df, k=6)

# Ask user for input once the program starts
print("Enter city names exactly as in 'name_en' column.")
start_city = input("Start city: ").strip()
end_city = input("End city: ").strip()

# Run Bellman-Ford shortest path
try:
    shortest_path_between(df, sparse_matrix_km, start_city, end_city)
except Exception as e:
    print("Error:", e)


Enter city names exactly as in 'name_en' column.

Shortest path from Madulsima to Akkaraipattu:
  1. Madulsima
  2. Arawakumbura
  3. Ayiwela
  4. Pitakumbura
  5. Nilgala
  6. Inginiyagala
  7. Pahalalanda
  8. Hingurana
  9. Irakkamam
  10. Navatkadu
  11. Akkaraipattu
Total distance: 85.752 km
